In [5]:
import numpy as np 
import pandas as pd

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# Файлы входных данных доступны только для чтения каталоге "../input/"


import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Вы можете записать до 20 ГБ в текущий каталог (/kaggle/working/), который сохраняется в качестве выходных данных при создании версии с помощью "Сохранить и запустить все".
# Вы также можете записать временные файлы в /kaggle/temp/, но они не будут сохранены вне текущего сеанса

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout

In [19]:
data = pd.read_csv("C:/Seversteel/train.csv")

In [34]:
l1=[]
l2=[] 
for img,ClassId,EncodedPixels in tqdm(data.values):
    image=cv2.imread("C:/Seversteel/train_images-20240227T134703Z-001/{}".format(img),cv2.IMREAD_GRAYSCALE)
    image=cv2.resize(image, (120,120))
    l1.append(image)
    l2.append(ClassId)

  0%|          | 0/7095 [00:00<?, ?it/s]


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
encoder = LabelEncoder()

X= np.array(l1)
X = X/255

y = encoder.fit_transform(l2)
y = to_categorical(y)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,shuffle=True)

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(120,120,1),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(256,activation="relu"))
model.add(Dense(4,activation="softmax"))

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,min_delta=0.001,restore_best_weights=True)

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(),
             metrics=["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=128,
                    verbose=1, callbacks=[early_stopping])

In [ ]:
def rle_encode(mask):
    rle_encoded_mask = []
    prev_value = mask[0][0]  # Access first element
    count = 1

    for value in mask[1:]:
        if value.any() != prev_value:  # Check if any element in value differs from prev_value
            rle_encoded_mask.append(int(prev_value))
            prev_value = value[0]  # Access first element for next comparison
            count = 1
        else:
            count += 1

        rle_encoded_mask.append(int(prev_value))
    return ' '.join(map(str, rle_encoded_mask))